In [1]:
import pandas as pd
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 200)
import numpy as np
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import pickle


## Step 1: Read in hold out data, scalers, and best model

In [2]:
#holdout = pd.read_csv('resources/movies_holdout_features.csv', index_col=0)
holdout = pd.read_csv('kc_house_data_test_features.csv', index_col=0)

In [3]:
# final_scaler = read_pickle(filename)
# final_model = read_pickle(filename)
file = open('model.pickle', 'rb')
final_model = pickle.load(file)

file = open('scaler.pickle','rb')
final_scaler = pickle.load(file)

# final_scaler = open('scaler.pickle')

In [4]:
holdout['bedrooms']=holdout['bedrooms'].replace([33],3)
holdout.loc[holdout['bedrooms'] >= 10]
holdout.loc[holdout['bathrooms'] == 0] 
holdout['bathrooms'] = holdout['bathrooms'].replace([0],.25)

## Step 2: Feature Engineering for holdout set

In [5]:
features = ['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15']

holdout_features = holdout[features]

In [6]:
holdout = holdout.drop(['sqft_living',], axis = 1)

In [7]:
holdout['multilevel'] = np.where(holdout['floors']> 1, 1, 0)
holdout['multilevel'].value_counts()
holdout['basement'] = np.where(holdout['sqft_basement']> 1, 1, 0)

In [8]:
holdout.replace({'sqft_lot': {1105: 1200}}, inplace=True)

In [9]:
holdout['outdoor_space'] = holdout['sqft_lot'] - (holdout['sqft_above']/holdout['floors'])
holdout['modern'] = np.where((holdout['yr_built']>= 2004)|(holdout['yr_renovated']>=2001), 1, 0)
holdout = holdout.drop(['id', 'date','sqft_lot', 'floors', 'sqft_basement', 'grade', 'sqft_lot15', 'sqft_living15'], axis=1)
holdout['outdoor_space^2'] = holdout['outdoor_space']*holdout['outdoor_space']
holdout['sqft_view'] = holdout['sqft_above']*holdout['view']
holdout['sqft_multilevel'] = holdout['sqft_above']*holdout['multilevel']
holdout['outdoor_modern'] = holdout['outdoor_space']*holdout['modern']
holdout['outdoorspace_waterfront'] = holdout['outdoor_space']*holdout['waterfront']
holdout = holdout.drop(['sqft_view','sqft_multilevel','outdoor_space^2'], axis=1)
holdout_dum = pd.get_dummies(holdout['zipcode'])
holdout = pd.concat([holdout, holdout_dum], axis=1)
holdout = holdout.drop(['outdoor_space', 'outdoor_modern', 'outdoorspace_waterfront'], axis=1)

In [10]:
holdout.shape

(4323, 84)

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.

In [12]:
selected = [    'bedrooms',    'bathrooms',   'waterfront',         'view',
          'condition',   'sqft_above',     'yr_built', 'yr_renovated',
            'zipcode',          'lat',         'long',   'multilevel',
           'basement',       'modern',          98001,          98002,
                98003,          98004,          98005,          98006,
                98007,          98008,          98010,          98011,
                98014,          98019,          98022,          98023,
                98024,          98027,          98028,          98029,
                98030,          98031,          98032,          98033,
                98034,          98038,          98039,          98040,
                98042,          98045,          98052,          98053,
                98055,          98056,          98058,          98059,
                98065,          98070,          98072,          98074,
                98075,          98077,          98092,          98102,
                98103,          98105,          98106,          98107,
                98108,          98109,          98112,          98115,
                98116,          98117,          98118,          98119,
                98122,          98125,          98126,          98133,
                98136,          98144,          98146,          98148,
                98155,          98166,          98168,          98177,
                98178,          98188,          98198,          98199]

In [ ]:
holdout.describe()

In [ ]:
scaled_data = final_scaler.transform(holdout)

In [ ]:
# transformed_holdout = final_scaler(holdout)

sc_features = pd.DataFrame(data=scaled_data, columns=holdout.columns)

## Step 3: Predict the holdout set

In [14]:
# final_answers = final_model.predict(transformed_holdout)
final_answers = final_model.predict(holdout)
final_answers

array([548253.75, 548253.75, 407399.25, ..., 288142.5 , 396938.5 ,
       288798.5 ])

## Step 4: Export your predictions

In [16]:
# final_answer.to_csv('housing_preds_your_name.csv')
pd.DataFrame(final_answers).to_csv('housing_preds_alanratliff.csv')